In [3]:
#1
#=
sneasy_param.ipynb: from slr data from Sneasy Brick, downscale segments in the US Gulf coast then run them through 
CIAM, creating CSV's of data output including net present value optimal, optimal cost,
no adapt cost, and optimal adaptation strategy
run this for 10000 trials of parameters, save parameter data
for the outputs, the rows are the segements, and the columns are typically models/trials from Sneasy's data
=#


using MimiCIAM
using Mimi
using PlotlyJS
using NetCDF
using CSV
using DataFrames
using Distributions

In [4]:
#2
include("../downscaling.jl")
include("../run_ciam_data_function.jl")


run_ciam_data_f (generic function with 3 methods)

Reading data

In [5]:
#4
ais_26_s = CSV.read("projections_csv/RCP26/projections_antarctic_RCP26_sneasybrick.csv", DataFrame)
ais_45_s = CSV.read("projections_csv/RCP45/projections_antarctic_RCP45_sneasybrick.csv", DataFrame)
ais_60_s = CSV.read("projections_csv/RCP60/projections_antarctic_RCP60_sneasybrick.csv", DataFrame)
ais_85_s = CSV.read("projections_csv/RCP85/projections_antarctic_RCP85_sneasybrick.csv", DataFrame)

gsic_26_s = CSV.read("projections_csv/RCP26/projections_glaciers_RCP26_sneasybrick.csv", DataFrame)
gsic_45_s = CSV.read("projections_csv/RCP45/projections_glaciers_RCP45_sneasybrick.csv", DataFrame)
gsic_60_s = CSV.read("projections_csv/RCP60/projections_glaciers_RCP60_sneasybrick.csv", DataFrame)
gsic_85_s = CSV.read("projections_csv/RCP85/projections_glaciers_RCP85_sneasybrick.csv", DataFrame)

gis_26_s = CSV.read("projections_csv/RCP26/projections_greenland_RCP26_sneasybrick.csv", DataFrame)
gis_45_s = CSV.read("projections_csv/RCP45/projections_greenland_RCP45_sneasybrick.csv", DataFrame)
gis_60_s = CSV.read("projections_csv/RCP60/projections_greenland_RCP60_sneasybrick.csv", DataFrame)
gis_85_s = CSV.read("projections_csv/RCP85/projections_greenland_RCP85_sneasybrick.csv", DataFrame)

te_26_s = CSV.read("projections_csv/RCP26/projections_thermal_RCP26_sneasybrick.csv", DataFrame)
te_45_s = CSV.read("projections_csv/RCP45/projections_thermal_RCP45_sneasybrick.csv", DataFrame)
te_60_s = CSV.read("projections_csv/RCP60/projections_thermal_RCP60_sneasybrick.csv", DataFrame)
te_85_s = CSV.read("projections_csv/RCP85/projections_thermal_RCP85_sneasybrick.csv", DataFrame)

lws_26_s = CSV.read("projections_csv/RCP26/projections_landwater_storage_sl_RCP26_sneasybrick.csv", DataFrame)
lws_45_s = CSV.read("projections_csv/RCP45/projections_landwater_storage_sl_RCP45_sneasybrick.csv", DataFrame)
lws_60_s = CSV.read("projections_csv/RCP60/projections_landwater_storage_sl_RCP60_sneasybrick.csv", DataFrame)
lws_85_s = CSV.read("projections_csv/RCP85/projections_landwater_storage_sl_RCP85_sneasybrick.csv", DataFrame)

gmsl_26_s = CSV.read("projections_csv/RCP26/projections_gmsl_RCP26_sneasybrick.csv", DataFrame)
gmsl_45_s = CSV.read("projections_csv/RCP45/projections_gmsl_RCP45_sneasybrick.csv", DataFrame)
gmsl_60_s = CSV.read("projections_csv/RCP60/projections_gmsl_RCP60_sneasybrick.csv", DataFrame)
gmsl_85_s = CSV.read("projections_csv/RCP85/projections_gmsl_RCP85_sneasybrick.csv", DataFrame)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,-0.154001,-0.152378,-0.13832,-0.190903,-0.0909671,-0.107157,-0.107471,-0.175071,0.00329118,-0.124439,-0.241963,-0.191096,-0.0374594,-0.107977,-0.171806,-0.0406856,-0.132156,-0.0386427,-0.0776097,-0.157261,-0.196005,-0.139731,-0.171433,-0.219722,-0.0861081,-0.126549,-0.170693,-0.0727316,-0.138534,-0.132613,-0.0442634,-0.167135,-0.135552,-0.13552,-0.15154,-0.195733,-0.0503788,-0.137933,-0.056919,-0.110344,-0.174851,-0.121954,-0.0261881,-0.190627,-0.102319,-0.186949,-0.0985081,-0.219043,-0.125521,-0.0765,-0.192236,-0.113795,-0.123018,-0.135911,-0.0999893,-0.0635513,-0.125339,-0.161951,-0.143,-0.198579,-0.128625,-0.0947574,-0.183687,-0.0602605,-0.148709,-0.17422,-0.190984,-0.0113189,-0.206109,-0.141493,-0.164335,-0.24565,-0.071948,-0.135471,-0.180145,-0.0694726,-0.15152,-0.152766,-0.0279172,-0.129846,-0.128634,-0.144433,-0.151635,-0.170574,-0.109084,-0.179444,-0.118101,-0.183775,-0.138527,-0.197735,-0.122748,-0.0633126,-0.105614,-0.158245,-0.135082,-0.161152,-0.198097,-0.0942303,-0.0169109,-0.17072,⋯
2,-0.173539,-0.126757,-0.167549,-0.144453,-0.138944,-0.122211,-0.0880347,-0.182372,-0.0798365,-0.0993328,-0.17468,-0.13596,-0.104102,-0.133068,-0.12258,-0.10387,-0.124668,-0.115195,-0.148217,-0.193955,-0.145567,-0.181728,-0.105204,-0.13488,-0.105452,-0.163376,-0.140655,-0.0947886,-0.131438,-0.0729415,-0.090269,-0.172844,-0.182573,-0.171943,-0.1445,-0.168406,-0.101705,-0.185707,-0.108433,-0.155026,-0.118469,-0.0990595,-0.0828548,-0.134144,-0.14257,-0.156891,-0.181147,-0.209676,-0.0102027,-0.0733447,-0.177543,-0.157737,-0.0881197,-0.124282,-0.0906836,-0.111802,-0.164702,-0.168881,-0.142494,-0.139035,-0.134444,-0.142102,-0.146924,-0.0903349,-0.17394,-0.189283,-0.147293,-0.0909113,-0.20083,-0.128393,-0.0951324,-0.161506,-0.159845,-0.0795949,-0.181114,-0.0650037,-0.170889,-0.151751,-0.0732168,-0.11555,-0.129708,-0.119998,-0.114972,-0.171302,-0.103899,-0.113348,-0.098487,-0.137117,-0.106738,-0.177524,-0.191751,-0.0954137,-0.179396,-0.148827,-0.137466,-0.180131,-0.207205,-0.0676596,-0.0527101,-0.136927,⋯
3,-0.184835,-0.155199,-0.130034,-0.140501,-0.148099,-0.151268,-0.105485,-0.172311,-0.0764371,-0.113145,-0.199371,-0.187471,-0.0547421,-0.145373,-0.101868,-0.077927,-0.134934,-0.0892489,-0.109227,-0.160074,-0.190971,-0.145237,-0.117072,-0.151532,-0.121563,-0.144616,-0.154828,-0.0738653,-0.136788,-0.0541951,-0.0951703,-0.225584,-0.136964,-0.19739,-0.114792,-0.153573,-0.0889946,-0.16675,-0.135063,-0.152177,-0.162135,-0.0911389,-0.123353,-0.143582,-0.103151,-0.191191,-0.138561,-0.212993,-0.0740284,-0.0714797,-0.187682,-0.126082,-0.0842235,-0.106103,-0.0974829,-0.121622,-0.171157,-0.209023,-0.166238,-0.131618,-0.18495,-0.13683,-0.164666,-0.0708538,-0.189106,-0.136018,-0.185622,-0.109764,-0.206188,-0.159123,-0.1073,-0.12601,-0.146892,-0.0694727,-0.132867,-0.0538786,-0

In [6]:
#5
years_s = []
for i in range(1850, 2300)
    push!(years_s, i)
end
years_s

451-element Vector{Any}:
 1850
 1851
 1852
 1853
 1854
 1855
 1856
 1857
 1858
 1859
    ⋮
 2292
 2293
 2294
 2295
 2296
 2297
 2298
 2299
 2300

In [7]:
#6
comp_26_s = (years_s[151:350], Matrix(ais_26_s)[151:350, :], Matrix(gsic_26_s)[151:350, :], Matrix(gis_26_s)[151:350, :], Matrix(te_26_s)[151:350, :], Matrix(lws_26_s)[151:350, :], Matrix(gmsl_26_s)[151:350, :])
comp_45_s = (years_s[151:350], Matrix(ais_45_s)[151:350, :], Matrix(gsic_45_s)[151:350, :], Matrix(gis_45_s)[151:350, :], Matrix(te_45_s)[151:350, :], Matrix(lws_45_s)[151:350, :], Matrix(gmsl_45_s)[151:350, :])
comp_60_s = (years_s[151:350], Matrix(ais_60_s)[151:350, :], Matrix(gsic_60_s)[151:350, :], Matrix(gis_60_s)[151:350, :], Matrix(te_60_s)[151:350, :], Matrix(lws_60_s)[151:350, :], Matrix(gmsl_60_s)[151:350, :])
comp_85_s = (years_s[151:350], Matrix(ais_85_s)[151:350, :], Matrix(gsic_85_s)[151:350, :], Matrix(gis_85_s)[151:350, :], Matrix(te_85_s)[151:350, :], Matrix(lws_85_s)[151:350, :], Matrix(gmsl_85_s)[151:350, :])


(Any[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009  …  2190, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2199], [0.003568475575939025 0.0023496230725262544 … 0.0011961308207884365 0.0026841393843157783; 0.0023874803488068523 0.0018642127724811854 … 0.0022569410688024925 0.002687111784238036; … ; 1.7337918733960032 1.0759793689310837 … 0.6504331483701105 1.2496706286419603; 1.743637820287714 1.0864254579803856 … 0.6576981036992867 1.2595298333414888], [0.012126232438020046 0.016145607112018612 … 0.01651255797056682 0.014577790553246334; 0.014185049971868681 0.015030933595367123 … 0.014332823661283012 0.010919150295025002; … ; 0.42600727234555585 0.29446418157233045 … 0.3226782833341778 0.36212619634632953; 0.42817482167469556 0.2959996252694101 … 0.3240051585545623 0.3683543888140833], [0.0005097017216186828 0.00021134137765874524 … -0.00018817416415430955 0.000567818739648517; 0.0010858101947455602 0.001318770981534159 … 0.00012454760005189307 0.0005407267793038367; … 

In [8]:
#7
lonlat_gulf = []
function create_lonlat_tuples(lonlat_data)
    for i in range(1, size(lonlat_data, 1))
        push!(lonlat_gulf, (lonlat_data[i,1], lonlat_data[i,2]))
    end
end

create_lonlat_tuples (generic function with 1 method)

In [9]:
#8
gulf_segments = Matrix(CSV.read("../gulf_segments.csv", DataFrame))

177×1 Matrix{String31}:
 "UnitedStates4592"
 "UnitedStates4593"
 "UnitedStates4594"
 "UnitedStates4595"
 "UnitedStates4596"
 "UnitedStates4599"
 "UnitedStates4600"
 "UnitedStates4601"
 "UnitedStates4615"
 "UnitedStates4603"
 ⋮
 "UnitedStates4653"
 "UnitedStates4654"
 "UnitedStates5591"
 "UnitedStates5592"
 "UnitedStates5593"
 "UnitedStates5594"
 "UnitedStates5595"
 "UnitedStates5596"
 "UnitedStates5597"

In [11]:
#9
lonlat_gulf_list = Matrix(CSV.read("../lonlat_gulf_list.csv", DataFrame))
create_lonlat_tuples(lonlat_gulf_list)

Testing downscale with smaller segment file

In [128]:
#11
#testing with smaller segment file
ensembles_run = 1:10000
lonlat_test = lonlat_gulf[1:10]
segments_test = gulf_segments[1:10]


result_45_s_gulf_t = downscale_brick(comp_45_s, lonlat_test, ensembles_run, 2000, 2199, 10)
run_45_s_gulf_t = result_45_s_gulf_t[1]
lslr_45_s_gulf_t = run_45_s_gulf_t[:,:,1]
for i in range(2, length(segments_test))
    lslr_45_s_gulf_t = vcat(lslr_45_s_gulf_t, run_45_s_gulf_t[:,:,i])
end
CSV.write("lslr_45_downscale_sneasy_gulf_test.csv", DataFrame(lslr_45_s_gulf_t, :auto))

"lslr_45_downscale_sneasy_gulf_test.csv"

In [36]:
#12
lslr_45_s_gulf_t = Matrix(CSV.read("lslr_45_downscale_sneasy_gulf_test.csv", DataFrame))

100000×20 Matrix{Float64}:
 0.0  0.0327748  0.0656825  0.107939   …  2.31508   2.55334   2.78475
 0.0  0.0277928  0.0633695  0.0933073     0.626198  0.657333  0.687004
 0.0  0.0300052  0.0694709  0.113434      2.15447   2.3567    2.55667
 0.0  0.0321379  0.0682109  0.116568      2.20001   2.38775   2.57973
 0.0  0.0293177  0.062918   0.106256      0.894612  0.946435  0.992823
 0.0  0.0261152  0.0640378  0.109149   …  1.06846   1.13175   1.18401
 0.0  0.0242169  0.0570668  0.0900779     0.905094  0.960437  1.00412
 0.0  0.0339226  0.0652524  0.106727      0.633387  0.661437  0.693557
 0.0  0.0298352  0.065485   0.105365      2.3739    2.61154   2.84244
 0.0  0.0230413  0.0489919  0.0841285     0.827912  0.870003  0.912476
 0.0  0.0306619  0.0655434  0.108904   …  1.82091   1.98974   2.1538
 0.0  0.028915   0.062003   0.107752      1.12961   1.24382   1.46217
 0.0  0.0244536  0.0543215  0.0976346     1.73073   1.88947   2.05429
 ⋮                                     ⋱                    

In [32]:
#13
run_name = "Sneasy_Gulf_245_test"
subset_name = "C:/Users/krfek/Downloads/gitdemo/mimiCIAM_demo/gulf_segments_test.csv"

init_settings = Dict(
        :init_filename   => string("$run_name", "_init.csv"),
        :lslrfile        => "lsl_rcp85_p50.csv",
        :subset          => subset_name,
        :ssp             => "IIASAGDP_SSP2_v9_130219",
        :ssp_simplified  => 2
    )

MimiCIAM.write_init_file(run_name, "..", init_settings)


model_245_gulf_s_t = MimiCIAM.get_model(initfile="../Sneasy_Gulf_245_test_init.csv")
MimiCIAM.run(model_245_gulf_s_t)   




Distribution 

In [106]:
#15 (dont rerun!)
movefactor_d  = truncated(Normal(1,1),0.5,3)
dvbm_d        = truncated(Normal(5.376,2.688),0.0,Inf)
vslel_d       = truncated(Normal(0.47,0.15),0.0,Inf)
vslmult_d     = truncated(Normal(200,100),0.0,Inf)
wvel_d        = truncated(Normal(1.16, 0.46),0.0,Inf)
wvpdl_d       = truncated(Normal(0.47,0.12),0.0,1.0)

Truncated(Normal{Float64}(μ=0.47, σ=0.12); lower=0.0, upper=1.0)

In [142]:
#16 (dont rerun!)
CSV.write("movefactor_s.csv", DataFrame(hcat(rand(movefactor_d, 10000)), :auto))
CSV.write("dvbm_s.csv", DataFrame(hcat(rand(dvbm_d, 10000)), :auto))
CSV.write("vslel_s.csv", DataFrame(hcat(rand(vslel_d, 10000)), :auto))
CSV.write("vslmult_s.csv", DataFrame(hcat(rand(vslmult_d, 10000)), :auto))
CSV.write("wvel_s.csv", DataFrame(hcat(rand(wvel_d, 10000)), :auto))
CSV.write("wvpdl_s.csv", DataFrame(hcat(rand(wvpdl_d, 10000)), :auto))



"wvpdl_s.csv"

In [12]:
#17
move_factor_s = Matrix(CSV.read("movefactor_s.csv", DataFrame))
dvbm_s = Matrix(CSV.read("dvbm_s.csv", DataFrame))
vslel_s = Matrix(CSV.read("vslel_s.csv", DataFrame))
vslmult_s = Matrix(CSV.read("vslmult_s.csv", DataFrame))
wvel_s = Matrix(CSV.read("wvel_s.csv", DataFrame))
wvpdl_s = Matrix(CSV.read("wvpdl_s.csv", DataFrame))

10000×1 Matrix{Float64}:
 0.41671323309870717
 0.4821939726167196
 0.6237254399934304
 0.5053948117930075
 0.2958500370680113
 0.6246427002165509
 0.34763429372773286
 0.6690333421844861
 0.3762825256190186
 0.4473939417728137
 ⋮
 0.46608293317405347
 0.4254755047206367
 0.6182180415526795
 0.5963427144596648
 0.3918061599165659
 0.549363211642908
 0.41902066094074675
 0.3787435497564034
 0.4601614149303505

In [13]:
#18
params = [move_factor_s, dvbm_s, vslel_s, vslmult_s, wvel_s, wvpdl_s]

6-element Vector{Matrix{Float64}}:
 [1.3616907543094252; 2.71563850742377; … ; 1.1325443598832952; 0.8825423643085583;;]
 [1.9926451375371466; 4.150807733636418; … ; 8.419424359517823; 5.221219436781835;;]
 [0.3013050487881483; 0.5283196079473862; … ; 0.4133267896269778; 0.6223936423686056;;]
 [208.80264972285923; 136.09810674813542; … ; 97.30664265643591; 134.17843658074852;;]
 [1.0694361428473813; 1.6942784286877195; … ; 1.1500307171475552; 1.4865910127217257;;]
 [0.41671323309870717; 0.4821939726167196; … ; 0.3787435497564034; 0.4601614149303505;;]

test ciam with smaller data

In [39]:
#20
params_test = [move_factor_s[1:3], dvbm_s[1:3], vslel_s[1:3], vslmult_s[1:3], wvel_s[1:3], wvpdl_s[1:3]]

6-element Vector{Vector{Float64}}:
 [1.3616907543094252, 2.71563850742377, 1.4953319826602258]
 [1.3616907543094252, 2.71563850742377, 1.4953319826602258]
 [1.3616907543094252, 2.71563850742377, 1.4953319826602258]
 [1.3616907543094252, 2.71563850742377, 1.4953319826602258]
 [1.3616907543094252, 2.71563850742377, 1.4953319826602258]
 [0.41671323309870717, 0.4821939726167196, 0.6237254399934304]

In [42]:
#21
segments_test = Matrix(CSV.read("../gulf_segments_test.csv", DataFrame))

10×1 Matrix{String31}:
 "UnitedStates4569"
 "UnitedStates4592"
 "UnitedStates4593"
 "UnitedStates4594"
 "UnitedStates4595"
 "UnitedStates4596"
 "UnitedStates4599"
 "UnitedStates4600"
 "UnitedStates4601"
 "UnitedStates4615"

In [55]:
#22
data_245_gulf_s_t = run_ciam_data_f(model_245_gulf_s_t, 245, lslr_45_s_gulf_t, segments_test, 10000, 20, [1; 5; 10; 15; 19], params_test, 3)

(Union{Missing, Float64}[8.324731777900893e-5 0.00016055618522857015 … 0.0003249424044820551 0.00018433118761535235; 0.004616797492587578 0.0052164885585773485 … 0.009028083208864505 0.008140416883616395; … ; 0.0016966279975844233 0.0019161056974080057 … 0.003332443413753421 0.0030047173212566004; 0.012927580585243046 0.02049710741480213 … 0.03354963387893558 0.023871496613175495], Union{Missing, Float64}[0.0004639747624838094 0.0008905533558776865 … 0.0009009692166587421 0.0005102626274797858; 0.016054028510468804 0.0183778148183244 … 0.021266972701951683 0.018821335578335485; … ; 0.0059561417940729 0.006819087356000006 … 0.007906157153080212 0.0069967408824185276; 0.045775710043507514 0.07135610185838494 … 0.06594738646862855 0.047715396778700334], Union{Missing, Float64}[0.00035580452205255373 0.0005853709487377644 … 0.0007752312033357275 0.000505039327335693; 0.05410989446662053 0.09408416856800335 … 0.12382988167597003 0.07643430322034268; … ; 0.045551675669250474 0.07879486542000

Run all gulf segments

In [14]:
#24
ensembles_run = 1:10000

result_26_s_gulf = downscale_brick(comp_26_s, lonlat_gulf, ensembles_run, 2000, 2199, 10)
run_26_s_gulf = result_26_s_gulf[1]
lslr_26_s_gulf = run_26_s_gulf[:,:,1]
for i in range(2, length(gulf_segments))
    lslr_26_s_gulf = vcat(lslr_26_s_gulf, run_26_s_gulf[:,:,i])
end
CSV.write("lslr_26_downscale_sneasy_gulf.csv", DataFrame(lslr_26_s_gulf, :auto))

result_45_s_gulf = downscale_brick(comp_45_s, lonlat_gulf, ensembles_run, 2000, 2199, 10)
run_45_s_gulf = result_45_s_gulf[1]
lslr_45_s_gulf = run_45_s_gulf[:,:,1]
for i in range(2, length(gulf_segments))
    lslr_45_s_gulf = vcat(lslr_45_s_gulf, run_45_s_gulf[:,:,i])
end
CSV.write("lslr_45_downscale_sneasy_gulf.csv", DataFrame(lslr_45_s_gulf, :auto))

result_60_s_gulf = downscale_brick(comp_60_s, lonlat_gulf, ensembles_run, 2000, 2199, 10)
run_60_s_gulf = result_60_s_gulf[1]
lslr_60_s_gulf = run_60_s_gulf[:,:,1]
for i in range(2, length(gulf_segments))
    lslr_60_s_gulf = vcat(lslr_60_s_gulf, run_60_s_gulf[:,:,i])
end
CSV.write("lslr_60_downscale_sneasy_gulf.csv", DataFrame(lslr_60_s_gulf, :auto))

result_85_s_gulf = downscale_brick(comp_85_s, lonlat_gulf, ensembles_run, 2000, 2199, 10)
run_85_s_gulf = result_85_s_gulf[1]
lslr_85_s_gulf = run_85_s_gulf[:,:,1]
for i in range(2, length(gulf_segments))
    lslr_85_s_gulf = vcat(lslr_85_s_gulf, run_85_s_gulf[:,:,i])
end
CSV.write("lslr_85_downscale_sneasy_gulf.csv", DataFrame(lslr_85_s_gulf, :auto))

"lslr_85_downscale_sneasy_gulf.csv"

In [15]:
#25
lslr_26_s_gulf = Matrix(CSV.read("lslr_26_downscale_sneasy_gulf.csv", DataFrame))
lslr_45_s_gulf = Matrix(CSV.read("lslr_45_downscale_sneasy_gulf.csv", DataFrame))
lslr_60_s_gulf = Matrix(CSV.read("lslr_60_downscale_sneasy_gulf.csv", DataFrame))
lslr_85_s_gulf = Matrix(CSV.read("lslr_85_downscale_sneasy_gulf.csv", DataFrame))

1770000×20 Matrix{Float64}:
 0.0  0.0252765  0.0639127  0.109012   …  4.30233  4.75799  5.21503  5.65582
 0.0  0.0230202  0.0592376  0.0975785     2.45967  2.74076  3.02252  3.30897
 0.0  0.0342594  0.0714959  0.116678      4.13645  4.63548  5.13537  5.64941
 0.0  0.0348236  0.0722657  0.121926      3.98152  4.41224  4.83549  5.23153
 0.0  0.0338532  0.068564   0.106911      3.38116  3.73737  4.09441  4.44837
 0.0  0.033536   0.0671182  0.11326    …  3.78812  4.22639  4.67001  5.10645
 0.0  0.0280038  0.0586526  0.0977554     3.11895  3.46614  3.80591  4.13976
 0.0  0.0254675  0.0632913  0.0941817     2.46908  2.71036  2.94248  3.18068
 0.0  0.0277933  0.0646743  0.109653      4.61912  5.13486  5.65936  6.17029
 0.0  0.0238459  0.0553402  0.0911487     4.15669  4.63582  5.10577  5.56837
 ⋮                                     ⋱                             
 0.0  0.0275253  0.0555687  0.103505      3.13455  3.48943  3.84986  4.20606
 0.0  0.0269264  0.0708148  0.121172      3.36059  3.74

In [21]:
#26
run_name = "Sneasy_Gulf_126"
subset_name = "/Users/aewsma/codes/jup_jul_mimi_demo/mimiCIAM_demo/gulf_segments.csv"

init_settings = Dict(
        :init_filename   => string("$run_name", "_init.csv"),
        :lslrfile        => "lsl_rcp85_p50.csv",
        :subset          => subset_name,
        :ssp             => "IIASAGDP_SSP1_v9_130219",
        :ssp_simplified  => 1
    )

MimiCIAM.write_init_file(run_name, "..", init_settings)


model_126_gulf_s = MimiCIAM.get_model(initfile="../Sneasy_Gulf_126_init.csv")
MimiCIAM.run(model_126_gulf_s) 

data_126_gulf_s = run_ciam_data_f(model_126_gulf_s, 126, lslr_26_s_gulf, segments, 10000, 20, [1; 5; 10; 15; 19], params, 10000)

UndefVarError: UndefVarError: segments not defined

In [17]:
#27
run_name = "Sneasy_Gulf_245"
subset_name = "/Users/aewsma/codes/jup_jul_mimi_demo/mimiCIAM_demo/gulf_segments.csv"

init_settings = Dict(
        :init_filename   => string("$run_name", "_init.csv"),
        :lslrfile        => "lsl_rcp85_p50.csv",
        :subset          => subset_name,
        :ssp             => "IIASAGDP_SSP2_v9_130219",
        :ssp_simplified  => 2
    )

MimiCIAM.write_init_file(run_name, "..", init_settings)


model_245_gulf_s = MimiCIAM.get_model(initfile="../Sneasy_Gulf_245_init.csv")
MimiCIAM.run(model_245_gulf_s) 

data_245_gulf_s = run_ciam_data_f(model_245_gulf_s, 245, lslr_45_s_gulf, segments, 10000, 20, [1; 5; 10; 15; 19], params, 10000)

SystemError: SystemError: opening file "/Users/aewsma/.julia/packages/MimiCIAM/dbhiy/src/../data/subsets/C:/Users/krfek/Downloads/gitdemo/mimiCIAM_demo/gulf_segments.csv": No such file or directory

In [18]:
#28
run_name = "Sneasy_Gulf_460"
subset_name = "/Users/aewsma/codes/jup_jul_mimi_demo/mimiCIAM_demo/gulf_segments.csv"

init_settings = Dict(
        :init_filename   => string("$run_name", "_init.csv"),
        :lslrfile        => "lsl_rcp85_p50.csv",
        :subset          => subset_name,
        :ssp             => "IIASAGDP_SSP4_v9_130219",
        :ssp_simplified  => 4
    )

MimiCIAM.write_init_file(run_name, "..", init_settings)


model_460_gulf_s = MimiCIAM.get_model(initfile="../Sneasy_Gulf_460_init.csv")
MimiCIAM.run(model_460_gulf_s) 

data_460_gulf_s = run_ciam_data_f(model_460_gulf_s, 460, lslr_60_s_gulf, segments, 10000, 20, [1; 5; 10; 15; 19], params, 10000)

SystemError: SystemError: opening file "/Users/aewsma/.julia/packages/MimiCIAM/dbhiy/src/../data/subsets/C:/Users/krfek/Downloads/gitdemo/mimiCIAM_demo/gulf_segments.csv": No such file or directory

In [19]:
#29
run_name = "Sneasy_Gulf_585"
subset_name = "/Users/aewsma/codes/jup_jul_mimi_demo/mimiCIAM_demo/gulf_segments.csv"

init_settings = Dict(
        :init_filename   => string("$run_name", "_init.csv"),
        :lslrfile        => "lsl_rcp85_p50.csv",
        :subset          => subset_name,
        :ssp             => "IIASAGDP_SSP5_v9_130219",
        :ssp_simplified  => 5
    )

MimiCIAM.write_init_file(run_name, "..", init_settings)


model_585_gulf_s = MimiCIAM.get_model(initfile="../Sneasy_Gulf_585_init.csv")
MimiCIAM.run(model_585_gulf_s) 

data_585_gulf_s = run_ciam_data_f(model_585_gulf_s, 585, lslr_85_s_gulf, segments, 10000, 20, [1; 5; 10; 15; 19], params, 10000)

SystemError: SystemError: opening file "/Users/aewsma/.julia/packages/MimiCIAM/dbhiy/src/../data/subsets/C:/Users/krfek/Downloads/gitdemo/mimiCIAM_demo/gulf_segments.csv": No such file or directory